In [2]:
import pandas as pd

In [56]:
meteorite_data = pd.read_csv('meteorites.csv')

In [57]:
meteorite_data.head()

,name,id,name_type,class,mass,fall,year,lat,long,geolocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)"


In [58]:
meteorite_data.isnull().sum()

name              0
id                0
name_type         0
class             0
mass            131
fall              0
year            291
lat            7315
long           7315
geolocation    7315
dtype: int64

In [59]:
# Drop rows with missing values
print(meteorite_data.shape)
meteorite_data = meteorite_data.dropna()
print(meteorite_data.shape)

(45716, 10)
(38115, 10)


In [60]:
# Incorrect data points according to kaggle -> https://www.kaggle.com/datasets/nasa/meteorite-landings
meteorite_data = meteorite_data[(meteorite_data['year'] >= 860) & (meteorite_data['year'] <= 2016)]
meteorite_data = meteorite_data[(meteorite_data['long'] <= 180) & (meteorite_data['long'] >= -180)]
meteorite_data = meteorite_data[(meteorite_data['lat'] != 0) & (meteorite_data['lat'] != 0)]
meteorite_data.shape

(31705, 10)

In [61]:
meteorite_data.to_csv('meteorites_clean.csv', index=False)

In [62]:
import reverse_geocoder as rg

def get_country_from_coords(lat, lon):
    results = rg.search((lat, lon))
    country_name = results[0]["cc"]
    return country_name

meteorite_data['countrycode'] = meteorite_data.apply(lambda row: get_country_from_coords(row['lat'], row['long']), axis=1)

In [63]:
meteorite_data.head()

,name,id,name_type,class,mass,fall,year,lat,long,geolocation,countrycode
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)",DE
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)",DK
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)",CA
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)",MX
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)",AR


In [64]:
continent_mapping = pd.read_csv('continentmapping.csv')

In [65]:
continent_mapping.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [66]:
merged_data = pd.merge(meteorite_data, continent_mapping, how='left', left_on='countrycode', right_on='alpha-2')

# Rename the columns to more descriptive names
merged_data.rename(columns={'name_x': 'name','name_y': 'countryname', 'region': 'continent'}, inplace=True)

# Drop the 'alpha-2', 'alpha-3', 'country-code', 'iso_3166-2', 'region-code',
# 'sub-region-code', and 'intermediate-region-code' columns, since we don't need them
merged_data.drop(['alpha-2', 'alpha-3', 'country-code', 'iso_3166-2',
                  'region-code', 'sub-region-code', 'intermediate-region-code'],
                 axis=1, inplace=True)
merged_data.loc[merged_data['countryname'] == 'Antarctica', 'continent'] = 'Antarctica'

In [67]:
merged_data.to_csv('meteorites_clean.csv', index=False)

In [80]:
# 19 countries parsed without a countrycode. These are all in Namibia
meteorite_data_clean = merged_data
mask = meteorite_data_clean['countrycode'].isna()

meteorite_data_clean.loc[mask, 'countrycode'] = 'NA'
meteorite_data_clean.loc[mask, 'countryname'] = 'Namibia'
meteorite_data_clean.loc[mask, 'continent'] = 'Africa'
meteorite_data_clean.loc[mask, 'sub-region'] = 'Sub-Saharan Africa'

In [83]:
meteorite_data_clean['countrycode'].isna().sum()

0

In [84]:
meteorite_data_clean.to_csv('meteorites_clean.csv', index=False)

In [24]:
# Change the year from float to int so that vega lite can recognise it
meteorite_data_clean = pd.read_csv('meteorites_clean.csv')
meteorite_data_clean['year'] = pd.to_numeric(meteorite_data_clean['year'], downcast='unsigned', errors='coerce')
meteorite_data_clean.to_csv('meteorites_clean_2.csv', index=False)

In [25]:
meteorite_data_clean.head()

,name,id,name_type,class,mass,fall,year,lat,long,geolocation,countrycode,countryname,continent,sub-region,intermediate-region
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",DE,Germany,Europe,Western Europe,NaN
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",DK,Denmark,Europe,Northern Europe,NaN
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",CA,Canada,Americas,Northern America,NaN
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,"(16.88333, -99.9)",MX,Mexico,Americas,Latin America and the Caribbean,Central America
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,"(-33.16667, -64.95)",AR,Argentina,Americas,Latin America and the Caribbean,South America


In [32]:
# Define a list of meteorite names to extract
meteorite_types = ['Iron, IVA', 'Iron, IAB-sLL', 'Iron, ungrouped', 'Iron, IIIAB', 'Iron, IID',
                   'Iron, IIE', 'Iron, IAB-sHL', 'Iron', 'Iron, IAB-ung', 'Iron, IIIAB-an',
                   'Iron, IAB complex', 'Iron, IIIE-an', 'Iron, IID-an', 'Iron, IIIE', 'Iron, IVA-an']
# Extract meteorites with the specified names
extracted_df = meteorite_data_clean[meteorite_data_clean['class'].isin(meteorite_types)]
extracted_df = extracted_df[extracted_df['year'] > 1800]

extracted_df.to_csv('iron_meteorites.csv', index=False)

In [ ]:
#Take a sample of 1000 for performance issues when coding
meteorite_data_clean = pd.read_csv('meteorites_clean_2.csv')
meteorite_data_clean = meteorite_data_clean.sample(n=1000)
meteorite_data_clean.to_csv('meteorites_clean_small.csv', index=False)